# Tratamento dos dados

In [ ]:
#imports
import pandas as pd
import numpy as np
import matplotlib as plt

In [ ]:
# Carregando os arquivos
load_file = "dados_compras.json"
purchase_file = pd.read_json(load_file, orient = "records")
purchase_file.head()

# Informações dos Consumidores
* Número total de consumidores

In [ ]:
display

# Análise Geral de Compras
* Número de itens exclusivos
* Preço médio de compra
* Número total de compras
* Rendimento total

# Informações Demográficas Por Gênero

* Porcentagem e contagem de compradores masculinos
* Porcentagem e contagem de compradores do sexo feminino
* Porcentagem e contagem de outros / não divulgados

# Análise de Compras Por Gênero 

* Número de compras
* Preço médio de compra
* Valor Total de Compra
* Compras for faixa etária

# Identifique os 5 principais compradores pelo valor total de compra e, em seguida, liste (em uma tabela):

* Login
* Número de compras
* Preço médio de compra
* Valor Total de Compra
* Itens mais populares

#  Identifique os 5 itens mais populares por contagem de compras e, em seguida, liste (em uma tabela):

* ID do item
* Nome do item
* Número de compras
* Preço do item
* Valor Total de Compra
* Itens mais lucrativos

# Identifique os 5 itens mais lucrativos pelo valor total de compra e, em seguida, liste (em uma tabela):

* ID do item
* Nome do item
* Número de compras
* Preço do item
* Valor Total de Compra